# Ray RLlib 강화학습 실험

이 노트북은 Ray RLlib을 사용한 강화학습 연구의 기본 템플릿입니다.

In [1]:
# 필수 라이브러리 import
import ray
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.sac.torch.default_sac_torch_rl_module import DefaultSACTorchRLModule
from ray.rllib.connectors.env_to_module import EnvToModulePipeline
from ray.rllib.connectors.module_to_env import ModuleToEnvPipeline
from ray.rllib.core.columns import Columns
from ray.rllib.core.rl_module.rl_module import RLModule, RLModuleSpec
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.env.single_agent_episode import SingleAgentEpisode
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os
import sys
import torch

PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
sys.path.append(PROJECT_ROOT)

# 패키지 Util 함수
from utils.utils import * 

print("라이브러리 import 완료")

라이브러리 import 완료


## 1. Ray 초기화

In [2]:
# Ray 초기화
# 로컬 모드로 실행 (디버깅용)
# ray.init(local_mode=True)

# 일반 모드로 실행 (실험용)
ray.init(ignore_reinit_error=True)

print(f"Ray 클러스터 정보: {ray.cluster_resources()}")

2026-01-16 16:44:21,102	INFO worker.py:2007 -- Started a local Ray instance.


Ray 클러스터 정보: {'node:10.254.233.114': 1.0, 'GPU': 1.0, 'CPU': 32.0, 'accelerator_type:G': 1.0, 'memory': 182405512397.0, 'object_store_memory': 78173791027.0, 'node:__internal_head__': 1.0}


/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/_private/worker.py:2046: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


## 2. 환경 설정

**참고**: 실제 환경은 별도 프로젝트로 관리되므로, 여기서는 예시로 Gymnasium 환경을 사용합니다.
실제 환경을 사용할 때는 해당 환경의 import 경로를 수정하세요.

In [3]:
ENV_NAME = "horcrux_env/plane-v0"

# 환경 정보 확인
import gymnasium as gym
import horcrux_env
from horcrux_env.envs import PlaneJoyDirWorld

env = gym.make(ENV_NAME)
print(f"환경: {ENV_NAME}")
print(f"관찰 공간: {env.observation_space}")
print(f"행동 공간: {env.action_space}")

ENV_OBS_SPACE = env.observation_space
ENV_ACTION_SPACE = env.action_space


env.close()

환경: horcrux_env/plane-v0
관찰 공간: Box(-inf, inf, (97,), float32)
행동 공간: Box(0.0, 2.7, (14,), float32)


In [4]:
ENV_CONFIG = {
    "forward_reward_weight": 175.0,
    "rotation_reward_weight": 100.0,
    "unhealthy_max_steps": 80.0,
    "healthy_reward": 3.0,
    "healthy_roll_range": (-40,40),
    "terminating_roll_range": (-80,80),
    "rotation_norm_cost_weight": 7.5,
    "termination_reward": 0,
    "gait_params": (30, 30, 40, 40, 0, 1),
    "use_friction_chg": True,
    "joy_input_random": False,
    "use_imu_window": True,
    "use_vels_window": True,
    "ctrl_cost_weight": 0.05,
}

RENDER_ENV_CONFIG = ENV_CONFIG.copy()
RENDER_ENV_CONFIG['render_mode'] = 'rgb_array'
RENDER_ENV_CONFIG['render_camera_name'] = 'ceiling'

# env = gym.make(ENV_NAME, **RENDER_ENV_CONFIG)

from ray.tune.registry import register_env
register_env(ENV_NAME, lambda config: PlaneJoyDirWorld( forward_reward_weight=ENV_CONFIG["forward_reward_weight"], 
                                                     rotation_reward_weight=ENV_CONFIG["rotation_reward_weight"], 
                                                     unhealthy_max_steps=ENV_CONFIG["unhealthy_max_steps"],
                                                     healthy_reward=ENV_CONFIG["healthy_reward"], 
                                                     healthy_roll_range=ENV_CONFIG["healthy_roll_range"],
                                                     terminating_roll_range=ENV_CONFIG["terminating_roll_range"],
                                                     rotation_norm_cost_weight=ENV_CONFIG["rotation_norm_cost_weight"],
                                                     termination_reward=ENV_CONFIG["termination_reward"],
                                                     gait_params=ENV_CONFIG["gait_params"],
                                                     use_friction_chg=ENV_CONFIG["use_friction_chg"],
                                                     joy_input_random=ENV_CONFIG["joy_input_random"],
                                                     use_imu_window=ENV_CONFIG["use_imu_window"],
                                                     ctrl_cost_weight=ENV_CONFIG["ctrl_cost_weight"],
                                                   )
            )

## 3. RLlib 알고리즘 설정

In [5]:
# PPO 알고리즘 설정 (다른 알고리즘으로 변경 가능: DQN, A3C, SAC 등)
config = (
    SACConfig()
    .environment(env=ENV_NAME)
    .training(
        gamma=0.90,
        model={
            "fcnet_hiddens": [512, 512, 512, 512, 512, 512, 48],
            "fcnet_activation": "relu",
            "vf_share_layers": False,
        },
        q_model_config={
            "fcnet_hiddens": [512, 512, 512, 512, 512, 512, 48],
            "fcnet_activation": "relu",
            "vf_share_layers": False,
        },
        policy_model_config={
            "fcnet_hiddens": [512, 512, 512, 512, 512, 512, 48],
            "fcnet_activation": "relu",
            "vf_share_layers": False,
        },
        train_batch_size=16384,
        train_batch_size_per_learner=8192,
        optimization_config={
            "actor_learning_rate": 3e-5,
            "critic_learning_rate": 3e-5,
            "entropy_learning_rate": 3e-5,
        },
        n_step=5,
        target_entropy="auto",
        replay_buffer_config={
            "capacity": 500000,
        },
    )
    .resources(num_gpus=1)  # GPU 사용 시 1로 변경
    .framework("torch")  # "tf2" 또는 "torch"
    .env_runners(
        num_env_runners=16,
        num_envs_per_env_runner=1,
    )
    .learners(
        num_learners=1,
        num_gpus_per_learner=1,
    )
    .rl_module(
        # rl_module_spec= RLModuleSpec(
        #     module_class=DefaultSACTorchRLModule,
        #     observation_space=ENV_OBS_SPACE,
        #     action_space=ENV_ACTION_SPACE,
        # ),
        model_config=DefaultModelConfig(
            fcnet_hiddens=[512, 512, 512, 512, 512, 512, 48],
            fcnet_activation="relu",
            vf_share_layers=False,
        ),
    )
)

print("알고리즘 설정 완료")
print(config)

알고리즘 설정 완료


## 4. 알고리즘 인스턴스 생성 및 학습

In [6]:
# 알고리즘 인스턴스 생성
algo = config.build_algo()

print("알고리즘 인스턴스 생성 완료")

2026-01-16 16:44:25,861	WARNING algorithm_config.py:5118 -- You are running SAC on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2026-01-16 16:44:25,862	WARNING algorithm_config.py:5147 -- You configured a custom `model` config (probably through calling config.training(model=..), whereas your config uses the new API stack! In order to switch off the new API stack, set in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. If you DO want to use the new API stack, configure your model, instead, through: `config.rl_module(model_config={..})`.
2026-01-16 16:44:25,862	WARNING sac.py:491 -- You are running SAC on the new API stack! This is the new default

알고리즘 인스턴스 생성 완료


In [7]:
# 학습 실행
NUM_ITERATIONS = 10000  # 학습 반복 횟수

import pprint

for i in range(NUM_ITERATIONS):
    result = algo.train()
    
    if (i + 1) % 150 == 0:
        print(f"Iteration {i + 1}")
        print(algo.save(algo.logdir))

print(algo.save(algo.logdir))
print("학습 완료")

/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
(pid=gcs_server) [2026-01-16 16:44:50,192 E 201418 201418] (gcs_server) gcs_server.cc:303: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(raylet) [2026-01-16 16:44:51,021 E 201580 201580] (raylet) main.cc:1032: Failed to establish connection to the metrics exporter agent. Metrics will not be expor

Iteration 150
TrainingResult(checkpoint=Checkpoint(filesystem=local, path=/home/bong/ray_results/SAC_horcrux_env-plane-v0_2026-01-16_16-44-25hskugxr4), metrics={'timers': {'training_iteration': 3.9343356280005537, 'restore_env_runners': 7.40291943657212e-06, 'training_step': 1.8166025631227602, 'env_runner_sampling_timer': 0.028656448571891816, 'replay_buffer_add_data_timer': 0.013374729179158749, 'replay_buffer_sampling_timer': 0.41186511042906204, 'learner_update_timer': 1.288758144157291, 'replay_buffer_update_prios_timer': 0.041310748443029295, 'synch_weights': 0.007154060171196761, 'synch_env_connectors': 0.0026488670009712223}, 'env_runners': {'module_to_env_connector': {'timers': {'connectors': {'listify_data_for_vector_env': np.float64(2.762213893862577e-05), 'get_actions': np.float64(0.00034585558864552593), 'un_batch_to_individual_items': np.float64(1.741324182238926e-05), 'remove_single_ts_time_rank_from_batch': np.float64(1.914245606256361e-06), 'normalize_and_clip_actions'

/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: Runti

Iteration 300
TrainingResult(checkpoint=Checkpoint(filesystem=local, path=/home/bong/ray_results/SAC_horcrux_env-plane-v0_2026-01-16_16-44-25hskugxr4), metrics={'timers': {'training_iteration': 4.367190215001756, 'restore_env_runners': 7.965527620399371e-06, 'training_step': 2.4836421978514407, 'env_runner_sampling_timer': 0.029944865109064266, 'replay_buffer_add_data_timer': 0.014083505729831813, 'replay_buffer_sampling_timer': 0.7226380662287193, 'learner_update_timer': 1.637499427847033, 'replay_buffer_update_prios_timer': 0.04533022774026904, 'synch_weights': 0.007899559110082919, 'synch_env_connectors': 0.003325148001749767}, 'env_runners': {'module_to_env_connector': {'timers': {'connectors': {'listify_data_for_vector_env': np.float64(2.895001297843991e-05), 'get_actions': np.float64(0.0003587355371786767), 'un_batch_to_individual_items': np.float64(1.724786373212798e-05), 'remove_single_ts_time_rank_from_batch': np.float64(1.959625553674907e-06), 'normalize_and_clip_actions': np

/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: Runti

...

/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: Runti

Iteration 9900
TrainingResult(checkpoint=Checkpoint(filesystem=local, path=/home/bong/ray_results/SAC_horcrux_env-plane-v0_2026-01-16_16-44-25hskugxr4), metrics={'timers': {'training_iteration': 6.294592641002964, 'restore_env_runners': 8.013252809178085e-06, 'training_step': 3.257280716551904, 'env_runner_sampling_timer': 0.03701593005709583, 'replay_buffer_add_data_timer': 0.03434566063340753, 'replay_buffer_sampling_timer': 1.6571075991910766, 'learner_update_timer': 1.403912250206049, 'replay_buffer_update_prios_timer': 0.07093397805525456, 'synch_weights': 0.013403083981538657, 'synch_env_connectors': 0.012500925004133023}, 'env_runners': {'module_to_env_connector': {'timers': {'connectors': {'listify_data_for_vector_env': np.float64(2.8667573208355308e-05), 'get_actions': np.float64(0.0003424073602108847), 'un_batch_to_individual_items': np.float64(1.6517435545959946e-05), 'remove_single_ts_time_rank_from_batch': np.float64(1.9278080969614486e-06), 'normalize_and_clip_actions': n

/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: Runti

TrainingResult(checkpoint=Checkpoint(filesystem=local, path=/home/bong/ray_results/SAC_horcrux_env-plane-v0_2026-01-16_16-44-25hskugxr4), metrics={'timers': {'training_iteration': 6.0052714129997185, 'restore_env_runners': 7.72174826124683e-06, 'training_step': 2.9947632345350574, 'env_runner_sampling_timer': 0.033684803084906886, 'replay_buffer_add_data_timer': 0.03699217003508238, 'replay_buffer_sampling_timer': 1.6643565236710127, 'learner_update_timer': 1.1343045873515076, 'replay_buffer_update_prios_timer': 0.07392535923077959, 'synch_weights': 0.013334114992030664, 'synch_env_connectors': 0.011980711002252065}, 'env_runners': {'module_to_env_connector': {'timers': {'connectors': {'listify_data_for_vector_env': np.float64(2.7998036472508182e-05), 'get_actions': np.float64(0.00034049371116404907), 'un_batch_to_individual_items': np.float64(1.597802138160309e-05), 'remove_single_ts_time_rank_from_batch': np.float64(1.9597229561406946e-06), 'normalize_and_clip_actions': np.float64(6.

/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/metrics/stats/ema.py:124: RuntimeWarning: Mean of empty slice
  return np.nanmean(self._values_to_merge)


# 5. 학습 결과 평가

In [8]:
from ray.rllib.connectors.env_to_module import EnvToModulePipeline
from ray.rllib.connectors.module_to_env import ModuleToEnvPipeline

# 학습된 정책으로 평가 (새 API 스택 사용)
NUM_EVAL_EPISODES = 10
eval_results = []

# RLModule과 Connector 파이프라인 가져오기
rl_module = algo.get_module()

# EnvToModule과 ModuleToEnv 파이프라인 가져오기
# algo 객체에서 직접 가져올 수 있는 경우
try:
    # env_runner에서 connector 가져오기
    env_runner = algo.env_runner
    env_to_module = env_runner._connectors[0] if hasattr(env_runner, '_connectors') else None
    module_to_env = env_runner._connectors[1] if hasattr(env_runner, '_connectors') and len(env_runner._connectors) > 1 else None
except:
    env_to_module = None
    module_to_env = None

# 파이프라인을 직접 생성해야 하는 경우
if env_to_module is None or module_to_env is None:
    # 간단한 방법: RLModule만 사용하고 직접 처리
    from ray.rllib.core.columns import Columns
    
    env = gym.make(ENV_NAME, render_mode=None)
    device = getattr(rl_module, "device", next(rl_module.parameters()).device)
    
    for episode in range(NUM_EVAL_EPISODES):
        obs, info = env.reset()
        episode_reward = 0
        episode_length = 0
        done = False
        
        # Episode 객체 생성
        episode_obj = SingleAgentEpisode(
            observations=[obs],
            observation_space=env.observation_space,
            action_space=env.action_space,
        )
        
        # Option 1
        while not done:
            # 관찰을 텐서로 변환
            obs_batch = np.expand_dims(obs, 0).astype(np.float32)
            obs_tensor = torch.from_numpy(obs_batch).to(device)
            
            # forward_inference 사용 (평가 시에는 탐색 없음)
            rl_module_out = rl_module.forward_inference({"obs": obs_tensor})
            
            # ModuleToEnv 파이프라인이 없으므로 직접 처리
            # action_dist_inputs에서 액션 추출
            if Columns.ACTION_DIST_INPUTS in rl_module_out:
                action_dist_inputs = rl_module_out[Columns.ACTION_DIST_INPUTS]
            elif "action_dist_inputs" in rl_module_out:
                action_dist_inputs = rl_module_out["action_dist_inputs"]
            else:
                # 다른 키 확인
                action_dist_inputs = list(rl_module_out.values())[0]
            
            # action_dist_inputs 처리 (mean만 사용하거나 분포에서 샘플링)
            if isinstance(action_dist_inputs, torch.Tensor):
                dist_params = action_dist_inputs[0] if len(action_dist_inputs.shape) > 1 else action_dist_inputs
                
                if len(dist_params.shape) == 0 or dist_params.shape[0] == 14:
                    # mean만 있는 경우 (deterministic)
                    action_tensor = dist_params[:14] if len(dist_params) >= 14 else dist_params
                elif dist_params.shape[0] == 28:
                    # mean(14) + log_std(14) 구조
                    mean = dist_params[:14]
                    log_std = dist_params[14:]
                    std = torch.exp(log_std.clamp(-20, 2))
                    from torch.distributions import Normal
                    normal_dist = Normal(mean, std)
                    action_tensor = normal_dist.sample()
                else:
                    action_tensor = dist_params
                
                # 액션 공간에 맞게 클리핑
                action_low = torch.tensor(env.action_space.low, device=device, dtype=action_tensor.dtype)
                action_high = torch.tensor(env.action_space.high, device=device, dtype=action_tensor.dtype)
                
                
                bounded_action = torch.tanh(action_tensor[:14])
                # print(f'over:{(bounded_action[0:14] > 1).sum()}, below:{(bounded_action[0:14] < -1).sum()}, total:{(bounded_action[0:14] > 1).sum() + (bounded_action[0:14] < -1).sum()}')
                
                real_action = action_low + (0.5 * (bounded_action + 1.0) * (action_high - action_low))
                # print(real_action)

                action = real_action.detach().cpu().numpy()
            else:
                action = np.array(action_dist_inputs)
                if len(action.shape) > 1:
                    action = action[0]
                action = np.clip(action, env.action_space.low, env.action_space.high)
            
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            episode_reward += reward
            episode_length += 1
            
            if episode_length > 6000:
                done = True

            # Episode 업데이트
            episode_obj.add_env_step(
                obs,
                action,
                reward,
                terminated=terminated,
                truncated=truncated,
            )
        
        # Option 2
        # while not done:
        #     input_dict = env_to_module(
        #         episodes=[episode_obj],
        #         rl_module=rl_module,
        #         explore=False,
        #         shared_data={},
        #     )

        #     rl_module_out = rl_module.forward_inference(input_dict)

        #     to_env = module_to_env(
        #         batch=rl_module_out,
        #         episodes=[episode_obj],
        #         rl_module=rl_module,
        #         explore=False,
        #         shared_data={},
        #     )

        #     action = to_env.pop(Columns.ACTIONS)[0]
            
        #     obs, reward, terminated, truncated, info = env.step(action)
        #     done = terminated or truncated
        #     episode_reward += reward
        #     episode_length += 1
            
        #     # Episode 업데이트
        #     episode_obj.add_env_step(
        #         obs,
        #         action,
        #         reward,
        #         terminated=terminated,
        #         truncated=truncated,
        #     )
            

        print(episode_obj.get_return())
        eval_results.append({
            'episode': episode + 1,
            'reward': episode_reward,
            'length': episode_length,
        })
        
        if episode_obj.is_done:
            obs, info = env.reset()
            episode_obj = SingleAgentEpisode(
                observations=[obs],
                observation_space=env.observation_space,
                action_space=env.action_space,
            )
    
    env.close()
else:
    # 파이프라인이 있는 경우 예제 방식 사용
    env = gym.make(ENV_NAME, render_mode=None)
    
    obs, _ = env.reset()
    episode_obj = SingleAgentEpisode(
        observations=[obs],
        observation_space=env.observation_space,
        action_space=env.action_space,
    )
    
    num_episodes = 0
    while num_episodes < NUM_EVAL_EPISODES:
        shared_data = {}
        input_dict = env_to_module(
            episodes=[episode_obj],
            rl_module=rl_module,
            explore=False,
            shared_data=shared_data,
        )
        
        rl_module_out = rl_module.forward_inference(input_dict)
        
        to_env = module_to_env(
            batch=rl_module_out,
            episodes=[episode_obj],
            rl_module=rl_module,
            explore=False,
            shared_data=shared_data,
        )
        
        action = to_env.pop(Columns.ACTIONS)[0]
        obs, reward, terminated, truncated, _ = env.step(action)
        
        episode_obj.add_env_step(
            obs,
            action,
            reward,
            terminated=terminated,
            truncated=truncated,
            extra_model_outputs={k: v[0] for k, v in to_env.items()},
        )
        
        if episode_obj.is_done:
            eval_results.append({
                'episode': num_episodes + 1,
                'reward': episode_obj.get_return(),
                'length': len(episode_obj),
            })
            obs, info = env.reset()
            episode_obj = SingleAgentEpisode(
                observations=[obs],
                observation_space=env.observation_space,
                action_space=env.action_space,
            )
            num_episodes += 1
    
    env.close()


eval_df = pd.DataFrame(eval_results)
print("\n평가 결과:")
print(eval_df)
print(f"\n평균 보상: {eval_df['reward'].mean():.2f} ± {eval_df['reward'].std():.2f}")
print(f"평균 에피소드 길이: {eval_df['length'].mean():.2f} ± {eval_df['length'].std():.2f}")

# # 평가 결과 저장
# eval_df.to_csv(RESULTS_DIR / "evaluation_results.csv", index=False)

-3276.4039427115335
-9850.072188104015
-15673.59652114871
-300.93567270228556
-1116.6027071451263
-1227.2568665667268
-3745.1554650030926
-5398.887282483202
-3338.757733269208
-2879.1584435942264

평가 결과:
   episode        reward  length
0        1  -3276.403943     249
1        2  -9850.072188    1073
2        3 -15673.596521    1347
3        4   -300.935673      74
4        5  -1116.602707     187
5        6  -1227.256867     402
6        7  -3745.155465     282
7        8  -5398.887282     470
8        9  -3338.757733     291
9       10  -2879.158444     291

평균 보상: -4680.68 ± 4711.53
평균 에피소드 길이: 466.60 ± 411.29


## 7. 모델 저장 및 로드

In [ ]:
algo.save(algo.logdir)

In [ ]:
# 모델 저장
checkpoint_dir = algo.save()
print(f"모델 저장 완료: {checkpoint_dir}")

# 모델 로드 예시 (주석 처리)
# loaded_algo = Algorithm.from_checkpoint(checkpoint_dir)

## 8. 정리

In [ ]:
# 알고리즘 정리
algo.stop()

# Ray 종료 (필요한 경우)
# ray.shutdown()

print("정리 완료")